In [1]:
#import librosa
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
from random import shuffle
from tqdm.notebook import tqdm
import pandas as pd
from time import time

import pickle

from scipy.special import softmax
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn import svm,metrics,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

#from scipy.misc import imread, imresize

%matplotlib inline


In [2]:
IMG_SIZE = 224

num_labels = 6 
emotions_list = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
class_to_idx = {'Angry': 0, 'Disgust': 1, 'Fear': 2, 'Happy': 3, 'Neutral': 4, 'Sad': 5, 'Surprise': 6}
idx_to_class = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}


In [3]:
import pickle
model_name = 'enet_b0_8'
MODEL2EMOTIW_FEATURES=model_name+'_afew_extra.pickle' 

print(MODEL2EMOTIW_FEATURES)

with open(MODEL2EMOTIW_FEATURES, 'rb') as handle:
    filename2features_train,filename2features_val=pickle.load(handle)
print(len(filename2features_train),len(filename2features_val))


enet_b0_8_afew_extra.pickle
773 383


In [4]:

DATA_DIR=r'D:\Users\amira\Documents\datasets\emotions\AudioVideo\faces+'


In [5]:
def create_dataset(filename2features,data_dir):
    x = []
    y = []
    has_faces=[]
    ind=0
    for class_name in class_to_idx:
        for filename in tqdm(os.listdir(os.path.join(data_dir,class_name))):
            fn=os.path.splitext(filename)[0] # goes through files names
            if not fn in filename2features:
                continue
            features=filename2features[fn]
            total_features=None

            if len(features[0])!=0:
                cur_features=features[0][features[-1]==1]

            if len(cur_features)==0:
                print(class_name,': ',filename)
                has_faces.append(0)
                total_features=np.zeros_like(feature)
            else:
                has_faces.append(1)
                #mean_features=features.mean(axis=0)
                mean_features = (np.mean(cur_features, axis=0))
                std_features = (np.std(cur_features, axis=0))
                max_features = (np.max(cur_features, axis=0))
                min_features = (np.min(cur_features, axis=0))

                # join several features together
                feature = np.concatenate((mean_features, std_features, min_features, max_features), axis=None)                    
                #feature = np.concatenate((mean_features, std_features, min_features), axis=None)
                #feature = np.concatenate((mean_features, min_features, max_features), axis=None)
                #feature = np.concatenate((max_features, std_features), axis=None)
                #feature=max_features

                total_features=feature
            
            if total_features is not None:
                x.append(total_features)
                y.append(class_to_idx[class_name])
    x=np.array(x)
    y=np.array(y)
    has_faces=np.array(has_faces)
    print(x.shape,y.shape)
    return x,y,has_faces

x_train_enet, y_train_enet, has_faces_train = create_dataset(filename2features_train, os.path.join(DATA_DIR, 'Train_AFEW'))
x_test_enet, y_test_enet, has_faces_test = create_dataset(filename2features_val, os.path.join(DATA_DIR, 'Val_AFEW'))
from sklearn import svm,metrics,preprocessing

x_train_norm=preprocessing.normalize(x_train_enet,norm='l2')
x_test_norm=preprocessing.normalize(x_test_enet,norm='l2')

  0%|          | 0/133 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

...

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

(383, 5120) (383,)


## Пример того, что получаются слегка различные "вероятности"

In [15]:
lin_svc = svm.LinearSVC(C=1.1) 
   
lin_svc.fit(x_train_norm[has_faces_train==1], y_train_enet[has_faces_train==1])
y_pred = lin_svc.predict(x_test_norm)

#print("Accuracy:",metrics.accuracy_score(y_test_enet[has_faces_test==1], y_pred[has_faces_test==1]))
print("Complete accuracy:",metrics.accuracy_score(y_test_enet, y_pred))

Complete accuracy: 0.5926892950391645


In [16]:
linsvc_proba = []
linsvc_dist = lin_svc.decision_function(x_test_norm)
for i in range(len(linsvc_dist)):
  linsvc_proba.append(softmax(linsvc_dist[i]))

linsvc_proba = np.array(linsvc_proba)
print("Complete accuracy:",metrics.accuracy_score(y_test_enet, np.argmax(linsvc_proba, axis=1)))

Complete accuracy: 0.5926892950391645


In [17]:
linsvc_proba

array([[0.38935395, 0.10488827, 0.11039422, ..., 0.08164325, 0.14255736,
        0.10062161],
       [0.30495769, 0.14751365, 0.10925197, ..., 0.05570574, 0.13410408,
        0.14461146],
       [0.29457795, 0.08879195, 0.06053486, ..., 0.1918812 , 0.07452043,
        0.09925956],
       ...,
       [0.16545626, 0.11063648, 0.17656009, ..., 0.1933704 , 0.15645165,
        0.14687391],
       [0.08984647, 0.11856051, 0.11976888, ..., 0.18202317, 0.24164934,
        0.19865268],
       [0.08286713, 0.11130445, 0.23236521, ..., 0.16970015, 0.11028841,
        0.22866322]])

In [11]:
lin_svc = svm.LinearSVC(C=1.1) 
   
lin_svc.fit(x_train_norm[has_faces_train==1], y_train_enet[has_faces_train==1])
y_pred = lin_svc.predict(x_test_norm)

#print("Accuracy:",metrics.accuracy_score(y_test_enet[has_faces_test==1], y_pred[has_faces_test==1]))
print("Complete accuracy:",metrics.accuracy_score(y_test_enet, y_pred))

Complete accuracy: 0.5926892950391645


In [14]:
linsvc_proba = []
linsvc_dist = lin_svc.decision_function(x_test_norm)
for i in range(len(linsvc_dist)):
  linsvc_proba.append(softmax(linsvc_dist[i]))

linsvc_proba = np.array(linsvc_proba)
print("Complete accuracy:",metrics.accuracy_score(y_test_enet, np.argmax(linsvc_proba, axis=1)))

Complete accuracy: 0.5926892950391645


In [13]:
linsvc_proba

array([[0.38935559, 0.10488681, 0.11039256, ..., 0.08164369, 0.14255842,
        0.10062185],
       [0.30495962, 0.14751192, 0.10925089, ..., 0.05570589, 0.13410498,
        0.14461156],
       [0.29457849, 0.08879097, 0.06053415, ..., 0.1918821 , 0.07452105,
        0.09925926],
       ...,
       [0.16545614, 0.11063531, 0.17655797, ..., 0.19337207, 0.15645329,
        0.14687414],
       [0.08984678, 0.11855963, 0.11976792, ..., 0.18202375, 0.24165115,
        0.19865201],
       [0.08286765, 0.11130391, 0.23236463, ..., 0.16970058, 0.11028937,
        0.22866209]])

## CalibratedClassifierCV

In [26]:
from sklearn.calibration import CalibratedClassifierCV

lin_svc = svm.LinearSVC(C=1.1)
clf = CalibratedClassifierCV(lin_svc) 
clf.fit(x_train_norm[has_faces_train==1], y_train_enet[has_faces_train==1])
linsvc_proba = clf.predict_proba(x_test_norm)

print("Complete accuracy:",metrics.accuracy_score(y_test_enet, np.argmax(linsvc_proba, axis=1)))

Complete accuracy: 0.5822454308093995


In [27]:
print(linsvc_proba)

[[0.69542496 0.04699263 0.06342438 ... 0.0357516  0.10525061 0.03691953]
 [0.52709082 0.12013372 0.05838211 ... 0.0189576  0.10898037 0.1165605 ]
 [0.48626496 0.02816888 0.01068643 ... 0.23116884 0.01755679 0.04474281]
 ...
 [0.17688351 0.05664132 0.19903203 ... 0.27945737 0.15100569 0.13014106]
 [0.03272507 0.07995076 0.07252398 ... 0.21964068 0.35516383 0.23414965]
 [0.02274285 0.04658735 0.33213132 ... 0.18404434 0.06003161 0.34379498]]


In [29]:
from sklearn.calibration import CalibratedClassifierCV

lin_svc = svm.LinearSVC(C=1.1)
clf = CalibratedClassifierCV(lin_svc, method='isotonic') 
clf.fit(x_train_norm[has_faces_train==1], y_train_enet[has_faces_train==1])
linsvc_proba = clf.predict_proba(x_test_norm)

print("Complete accuracy:",metrics.accuracy_score(y_test_enet, np.argmax(linsvc_proba, axis=1)))

Complete accuracy: 0.597911227154047


In [30]:
print(linsvc_proba)

[[0.76185063 0.01945522 0.06440171 ... 0.01220487 0.12611788 0.01596968]
 [0.59674847 0.15358354 0.03821546 ... 0.         0.07955697 0.10872361]
 [0.56681922 0.01001122 0.         ... 0.25104857 0.00349553 0.03566358]
 ...
 [0.15726708 0.0125282  0.27367087 ... 0.21835835 0.18630157 0.15187394]
 [0.01612505 0.09408085 0.07935971 ... 0.22069199 0.32508118 0.26466122]
 [0.01272604 0.01645364 0.29021132 ... 0.2165723  0.02099834 0.44303836]]
